In [1]:
from kmeans.gensim_kmeans import KMeans
from preprocess import load_model
from pprint import pprint
import torch

In [2]:
corpus, dictionary, tfidf, lsi, mSimilar = load_model("./preprocess/model/")

2021-04-08 21:19:30,454 : INFO : loaded corpus index from ./preprocess/model/corpus.mm.index
2021-04-08 21:19:30,454 : INFO : initializing cython corpus reader from ./preprocess/model/corpus.mm
2021-04-08 21:19:30,454 : INFO : accepted corpus with 9804 documents, 255478 features, 6928768 non-zero entries
2021-04-08 21:19:30,454 : INFO : loading Dictionary object from ./preprocess/model/model.dic
2021-04-08 21:19:30,626 : INFO : Dictionary lifecycle event {'fname': './preprocess/model/model.dic', 'datetime': '2021-04-08T21:19:30.626017', 'gensim': '4.0.1', 'python': '3.9.2 (tags/v3.9.2:1a79785, Feb 19 2021, 13:44:55) [MSC v.1928 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'loaded'}
2021-04-08 21:19:30,626 : INFO : loading TfidfModel object from ./preprocess/model/model.tfidf
2021-04-08 21:19:30,861 : INFO : TfidfModel lifecycle event {'fname': './preprocess/model/model.tfidf', 'datetime': '2021-04-08T21:19:30.861345', 'gensim': '4.0.1', 'python': '3.9.2 (tags/v3.

In [3]:
X = lsi[tfidf[corpus]]

In [4]:
import numpy as np

X_array = np.array([[i[1] for i in doc] for doc in X])
X_torch = torch.from_numpy(X_array)

In [5]:
import torch

if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')


In [6]:
from file_utils import get_files

f_list = get_files("./corpus/train/")
l = np.array([int(i.split("\\")[-1].split("-")[0][1:]) for i in f_list])

In [17]:
max_acc = 0
max_i = -1
from training_utils import best_map

# 629
for i in range(3000, 4000):
    kmeans = KMeans(20, random_state=i, device=device)
    label, _ = kmeans.fit_predict(X, mSimilar, X_array, X_torch)
    acc, _ = best_map(l, label.numpy())
    if acc > max_acc:
        max_acc = acc
        max_i = i
    if acc > 0.50:
        print(i, acc)
print("max", max_i, max_acc)

max 3361 0.4971440228478172


In [16]:
from training_utils import best_map
kmeans = KMeans(20, random_state=629, device=device)
label, _ = kmeans.fit_predict(X, mSimilar, X_array, X_torch)
acc, _ = best_map(l, label.numpy())
print(acc)


0.5042839657282742
